In [2]:
import numpy as np
import cv2
import cv2 as cv
from FINAL_IMG_PREP import *
from video_info import *

In [3]:
path ='/Users/monica_air/Documents/Caba_fire_smoke_detection/surveilance_fire/Riverside cosplayer arrested  in Ontario arson fire; incident captured on surveillance video.mp4'

In [4]:
cap, width, height, fps, fourcc, filename, out = vid_info(path, 'XVID', '0514_test')

<VideoCapture 0x7f87da982c50>
cap <VideoCapture 0x7f87da982c50>, width 540.0, height 360.0, fps 30.008813958277152
codec {0} filename 0514_test.mp4


In [5]:
if not cap.isOpened():
    print('Video open failed!')
    sys.exit()

# 배경 영상 등록
ret, back = cap.read()

if not ret:
    print('Background image registration failed!')
    sys.exit()
    
#background 만들기
back=background(cap)
# 비디오 매 프레임 처리
while True:
    ret, frame = cap.read()
    
    if not ret:
        break
    
    # 현재 프레임 영상 그레이스케일 변환
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # 노이즈 제거
    gray = cv2.GaussianBlur(gray, (0, 0), 1.0)
    
    #####
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    h,s,v = cv2.split(hsv)
    
    # 차영상 구하기 $ 이진화
    # absdiff는 차 영상에 절대값
    diff = cv2.absdiff(gray, back)
    # gray_mean() or 밝기정도 지정 가능
    
    
    ####!!!!!back.mean()으로 하면 smoke까지 가능?
    _, diff = cv2.threshold(diff, v.mean(), 255, cv2.THRESH_BINARY)

    # 레이블링을 이용하여 바운딩 박스 표시
    cnt, _, stats, _ = cv2.connectedComponentsWithStats(diff)
    
    #contour
    contours, _ = cv2.findContours(diff, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    
    for i in range(1, cnt):
        x, y, w, h, s = stats[i]
        
        if s < 100:
            continue
        
#         cv2.drawContours(frame, contours, -1, (0, 0, 255),5)
        cv2.rectangle(frame, (x, y, w, h), (0, 255, 0), 1)
#         cv2.putText(frame, "Status:{}".format('!!FIRE!!'), (10,20),cv2.FONT_HERSHEY_SIMPLEX,
#                    1, (0,0,255),3)

###light detection
        ret, thr = cv2.threshold(gray,180, 255,cv2.THRESH_TOZERO)
        con, hier = cv2.findContours(thr, cv.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        for cnt in con:
            cv2.drawContours(frame, [cnt], -1, (0,0,255),1)
            
    cv2.imshow('frame', frame)
    out.write(frame)
    
    if cv2.waitKey(30) == 27:
        break

cap.release()
out.release()
cv2.destroyAllWindows()